## Preparation
### Import

In [19]:
import numpy as np
import pandas as pd
# %pip install -q amplpy gspread --upgrade
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin","highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics


AMPL Development Version 20240606 (MSVC 19.40.33811.0, 64-bit)
Demo license with maintenance expiring 20260131.
Using license file "c:\Users\ASUS\miniconda3\envs\dunnbebes\lib\site-packages\ampl_module_base\bin\ampl.lic".



In [20]:
%%ampl_eval
option version;

option version 'AMPL Development Version 20240606 (MSVC 19.40.33811.0, 64-bit)\
Demo license with maintenance expiring 20260131.\
Using license file "c:\Users\ASUS\miniconda3\envs\dunnbebes\lib\site-packages\ampl_module_base\bin\ampl.lic".\
';


### Load Data

In [21]:
demand = pd.read_csv("Demand.csv")
ProductID = sorted(demand['ProductID'].unique().tolist())
ShipToID  = sorted(demand['ShipToID'] .unique().tolist())

demand_df = (
    demand.groupby(["ShipToID", "ProductID"])["VolumeInKG"]
    .sum()
    .reset_index()
)

demand_dict = {(row['ShipToID'], row['ProductID']): round(row['VolumeInKG'],4) for index, row in demand_df.iterrows()}


## Model
### Mathematical model

In [22]:
%%writefile shipment.mod
reset;
# Set
set ShipToID;
set ProductID;

# Parameters
param VolumeInKG {ShipToID, ProductID} >= 0 default 0;

# Variables
var MetDemand {ShipToID, ProductID} >= 0;
var UnmetDemand {ShipToID, ProductID} >= 0;

# Objective
minimize Demand:
    sum {s in ShipToID, p in ProductID} UnmetDemand[s, p];

# Constraints
subject to Demand_Balance {s in ShipToID, p in ProductID}:
   MetDemand[s, p] + UnmetDemand[s, p] = VolumeInKG[s, p];
   

Overwriting shipment.mod


### Load Set and Parameters

In [23]:
ampl.read('shipment.mod')

# Load data into ampl
ampl.set['ShipToID'] = ShipToID
ampl.set['ProductID'] = ProductID
ampl.param['VolumeInKG'] = demand_dict

### Solve

In [24]:
%%ampl_eval
option solver highs;
solve;
# display MetDemand, UnmetDemand;

Solution determined by presolve;
objective Demand = 0.


# Output

In [25]:
MetDemand   = ampl.var['MetDemand']  .to_pandas()
UnmetDemand = ampl.var['UnmetDemand'].to_pandas()

result1 = MetDemand.reset_index()
result1.columns = ['ShipToID', 'ProductID','MetDemand']

result2 = UnmetDemand.reset_index()
result2.columns = ['ShipToID', 'ProductID','UnmetDemand']

demand_balance = pd.merge(result1, result2, on=['ShipToID', 'ProductID'], how='outer')



In [26]:
print(demand_balance)

        ShipToID     ProductID  MetDemand  UnmetDemand
0           2001   102371 - AU          0            0
1           2001   102372 - AU          0            0
2           2001   102377 - AU          0            0
3           2001   102378 - AU          0            0
4           2001   102380 - AU          0            0
...          ...           ...        ...          ...
340438      8012  3120063 - AU          0            0
340439      8012  3120064 - AU          0            0
340440      8012  3120066 - AU          0            0
340441      8012  3120067 - AU          0            0
340442      8012  3201208 - AU          0            0

[340443 rows x 4 columns]


In [27]:
result            = pd.ExcelWriter('checking.xlsx', engine='xlsxwriter')
demand_balance  .to_excel(result, sheet_name='demand_balance', index=False)
result.close() 
